# Digital Project for Data Science and Business Analytics

 > The primary objective of this project is to analyze how black-box machine learning models make predictions and determine what factors impact these predictions the most. Although it is important for models to make good predictions, this project is especially concerned with interpretability because it is important that the reasoning behind predictions is understood, especially in areas such as medicine, where decisions have serious consequences.

For this, various Explainable Artificial Intelligence (xAI) methods are used on stroke risk prediction models developed on the Stroke dataset, which is publicly available on Kaggle. The aim here is not only to produce explanations but also to critically examine the validity, reliability, and relevance of those explanations.


***

# Imports and Initial Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sn
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, precision_recall_curve
)
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
import collections
from collections import defaultdict
from lime.submodular_pick import SubmodularPick

In [ ]:
data = pd.read_csv("./healthcare-dataset-stroke-data.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
for col in data.select_dtypes(include="object").columns :
    data[col] = data[col].astype("category")
data = data.drop(columns=["id"])

In [ ]:
data.info()

| Feature             | Type               | Description                           | Possible Values / Format                                           | Explanation / Notes                                               |
| ------------------- | ------------------ | ------------------------------------- | ------------------------------------------------------------------ | -----------------------------------
| `gender`            | Categorical        | Biological sex of the patient         | `Male`, `Female`                                                   | Can be used as a feature; may need encoding for ML models         |
| `age`               | Numerical          | Age of the patient in years           | Float                                                              | Important risk factor for stroke; higher age often increases risk |
| `hypertension`      | Categorical/Binary | Whether patient has hypertension      | `0` (No), `1` (Yes)                                                | Hypertension is a key stroke risk factor                          |
| `heart_disease`     | Categorical/Binary | Whether patient has any heart disease | `0` (No), `1` (Yes)                                                | Another major risk factor for stroke                              |
| `ever_married`      | Categorical        | Marital status                        | `Yes`, `No`                                                        | Could correlate with lifestyle or social support                  |
| `work_type`         | Categorical        | Type of occupation                    | `Private`, `Self-employed`, `Govt_job`, `Children`, `Never_worked` | Can reflect lifestyle and stress levels                           |
| `Residence_type`    | Categorical        | Urban or rural living area            | `Urban`, `Rural`                                                   | Can affect access to healthcare and lifestyle patterns            |
| `avg_glucose_level` | Numerical          | Average blood glucose level (mg/dL)   | Float                                                              | High glucose/diabetes increases stroke risk                       |
| `bmi`               | Numerical          | Body Mass Index                       | Float (may contain NaN)                                            | Obesity is a risk factor; missing values may need imputation      |
| `smoking_status`    | Categorical        | Smoking habits                        | `never smoked`, `formerly smoked`, `smokes`, `Unknown`             | Smoking is a major risk factor; `Unknown` needs special handling  |
| `stroke`            | Categorical/Binary | Whether patient has had a stroke      | `0` (No), `1` (Yes)                                                | Target variable for prediction                                    |


In [ ]:
data.describe()

## Unvalidated values 

> The minimum value of the age column is 0.008 years , which is suspicious

In [ ]:
plt.title("Distribution of Age")
plt.ylabel("Age")
plt.hist(data["age"])

In [ ]:
data[data["age"]<=1]

> 💡 Upon examining the dataset, the small age values (e.g., 0.08, 0.16, 0.32 years) correspond to children under 1 year old.


# Data Cleaning & Feature Engineering + Visualizations

## NaN Values

In [ ]:
data.isnull().sum()

In [ ]:
plt.title("Data distribution before filling Nan")
plt.xlabel("BMI")
plt.ylabel("Frequency")
plt.hist(data["bmi"])
plt.show()

### Fill NaN with median since the data distribution is skewed

In [ ]:
data["bmi"] = data["bmi"].fillna(data["bmi"].median())

In [ ]:
plt.title("Data distribution after filling Nan")
plt.xlabel("BMI")
plt.ylabel("Frequency")
plt.hist(data["bmi"])
plt.show()

In [ ]:
data.isnull().sum()

## Duplicated Values

In [ ]:
print("Total number of duplicated rows: ", data.duplicated().sum())

## Detecting Outliers

In [ ]:
for col in data.drop(columns=["stroke"]).select_dtypes("number").columns:
    plt.figure()  
    plt.title(f"Data distribution of {col}")
    plt.hist(data[col])  
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
numeric_cols = data.drop(columns=["hypertension", "heart_disease", "stroke"]).select_dtypes(include="number").columns

for col in numeric_cols:
    plt.figure(figsize=(6, 4))  
    plt.title(f"Box plot of {col}")
    plt.boxplot(data[col])
    plt.xlabel(col)
    plt.show()


> 💡 Although there are some outliers in bmi and avg_glucose_level, We will primarily use tree-based models (e.g., Random Forest, XGBoost, CatBoost), which are naturally robust to outliers. Therefore, these extreme values will be retained, as they may carry important information for predicting stroke.

### Data Cleaning Based on Counterfactual Analysis

Upon examining the dataset and generated counterfactuals, we identified **two types of clinically implausible cases**:

1. **Extremely low glucose levels** – Some counterfactuals suggested that lowering glucose would increase stroke risk. This is **not supported by medical evidence** and occurs only in a small subset of patients (rare outliers).  

2. **Extremely high BMI values (≥ 50)** – A few instances indicated that very high BMI decreases stroke risk, which is **clinically unrealistic** and also represents a small portion of the dataset.  

Given that these cases are rare and potentially misleading, it is reasonable to remove them to **improve the plausibility and reliability** of subsequent analyses.


In [ ]:
# Remove rows where avg_glucose_level <= 70 AND stroke == 1
data = data[~((data['avg_glucose_level'] <= 70) & (data['stroke'] == 1))]
data = data[~(data['bmi'] >= 50)]

## Imbalanced target classes

In [ ]:
plt.title("Data distribution of stroke")
plt.hist(data["stroke"])
plt.show()

### So the target value is highly imbalanced. I will using SMOTE to oversampling the minority class.

In [ ]:
data.select_dtypes("number").corr()

# Building models

## Preprocessing

In [ ]:
category_mappings = {}
dataForModel = data.copy()
for col in ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']:
    dataForModel[col] = dataForModel[col].astype('category')
    category_mappings[col] = dict(enumerate(dataForModel[col].cat.categories))
    dataForModel[col] = dataForModel[col].cat.codes

X = dataForModel.drop(columns="stroke")
y = dataForModel["stroke"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
category_mappings

### SMOTE

In [ ]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

plt.hist(y_train_res)
plt.title("Target classes after using SMOTE")
plt.show()

### Function to calculate metrics

In [ ]:

def evaluate_model(y_true, y_pred, y_proba):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1-Score:", f1_score(y_true, y_pred))
    print("ROC-AUC:", roc_auc_score(y_true, y_proba))
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    plt.figure(figsize=(6,5))
    sn.heatmap(
        cm, 
        annot=True, 
        fmt="d", 
        cmap="Blues",
        xticklabels=["Predicted No Stroke", "Predicted Stroke"],
        yticklabels=["Actual No Stroke", "Actual Stroke"]
    )
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.title("Confusion Matrix")
    plt.show()
    
    
    print("\nClassification Report:\n", classification_report(y_true, y_pred))

## Random forest 

#### Tunning hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 500],      # number of trees
    'max_depth': [None, 5, 10, 20],      # max depth of tree
    'min_samples_split': [2, 5, 10],     # min samples to split
    'min_samples_leaf': [1, 2, 4],       # min samples per leaf
    'class_weight': ['balanced', {0:1,1:5}, {0:1,1:10}]  # emphasize minority
}


rf = RandomForestClassifier(random_state=42)


grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='recall',  # maximize recall for minority class
    cv=5,
    n_jobs=-1,
    verbose=2
)


grid_search.fit(X_train_res, y_train_res)

print("Best parameters found:", grid_search.best_params_)
print("Best recall score:", grid_search.best_score_)

best_rf = grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)
y_proba_rf = best_rf.predict_proba(X_test)[:,1]

print("===== Random Forest (Grid Search) =====")
evaluate_model(y_test, y_pred_rf, y_proba_rf)


#### Using tunned hyperparameters

In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=1,
    class_weight={0: 1, 1: 2},  
    random_state=42
)


rf.fit(X_train_res, y_train_res)


y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]


print("===== Random Forest =====")
evaluate_model(y_test, y_pred_rf, y_proba_rf)


In [ ]:
feature_importances = pd.DataFrame({
    'feature': X_train_res.columns,
    'importance': rf.feature_importances_
}).sort_values(by='importance', ascending=True)  # ascending for horizontal bar

# Plot horizontal bar chart
plt.figure(figsize=(10,6))
plt.barh(feature_importances['feature'], feature_importances['importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Random Forest Feature Importances')
plt.show()


## XGboost 

In [ ]:
scale_pos_weight = (y_train==0).sum() / (y_train==1).sum()  # original ratio
scale_pos_weight_tuned = scale_pos_weight * 1.5  # increase weight for minority
xgb = XGBClassifier(
    learning_rate=0.01,
    max_depth=3,
    n_estimators=1000,
    scale_pos_weight=scale_pos_weight_tuned,  # incresing weight for minority class
    use_label_encoder=False,
    random_state=42
)

xgb.fit(X_train_res, y_train_res)
y_pred_xgb = xgb.predict(X_test)
y_proba_xgb = xgb.predict_proba(X_test)[:, 1]

print("===== XGBoost =====")
evaluate_model(y_test, y_pred_xgb, y_proba_xgb)

## Catboost

In [ ]:
scale_pos_weight = (y_train==0).sum() / (y_train==1).sum()  # original ratio
scale_pos_weight_tuned = scale_pos_weight * 1.5  # increase weight for minority
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Prepare Pool object for CatBoost (categorical features are indexed)
cat_features_indices = [X_train_res.columns.get_loc(c) for c in categorical_cols]

catboost_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    eval_metric='Recall',
    scale_pos_weight=scale_pos_weight_tuned,# incresing weight for minority class
    random_state=42,
    verbose=100
)

catboost_model.fit(
    X_train_res, y_train_res,
    cat_features=cat_features_indices
)

y_pred = catboost_model.predict(X_test)
y_proba = catboost_model.predict_proba(X_test)[:,1]


print("----Catboost-------")
evaluate_model(y_test, y_pred, y_proba)


> 🌟 We observed that after removing those outliers(avg_glucose_level <=70 but stroke = 1) , we can we that our recall increase significantly 

> 💡 In conclusion, for healthcare applications, recall is prioritized over precision because false negatives are far more dangerous than false positives. Accepting lower precision is justified in order to ensure that critical cases are not missed.

# xAI Methods

***

# SHAP

While model performance metrics indicate how well a classifier performs, they do not explain why predictions are made. To address this limitation, we use SHAP (SHapley Additive exPlanations) to interpret the behavior of our stroke prediction model at both global and local levels. Unlike LIME, SHAP provides explanations that are theoretically grounded and consistent across different predictions.

In [ ]:
%pip install shap -q

import shap

In [ ]:
# Initialize SHAP explainer for catboost and xgboost models
explainer_xgb = shap.Explainer(xgb)

explainer_cat = shap.Explainer(catboost_model)

# Compute SHAP values for the test set
shap_values_xgb = explainer_xgb(X_test)

shap_values_cat = explainer_cat(X_test)

The SHAP explainer computes contribution values for each feature by comparing model predictions against a baseline (expected value). Each SHAP value represents how much a feature increases or decreases the prediction for a given instance.

### Global Explanation: Feature Importance

The purpose of global explaination is to understand which features most influence stroke predictions overall.

In [ ]:
shap.summary_plot(shap_values_xgb, X_test, show=False)
plt.title("SHAP Summary plot for XGBoost stroke prediction model")
plt.show()

In [ ]:
shap.summary_plot(shap_values_cat, X_test, show=False)
plt.title("SHAP Summary plot for CatBoost stroke prediction model")
plt.show()

The SHAP summary plots ranks features according to their average absolute contribution to the model output. Features at the top of the plot, such as age and average glucose level, have the greatest influence on predictions. The color gradient indicates whether high or low feature values increase or decrease predicted stroke risk.

📌 Key Insight

- Higher age and glucose levels consistently increase predicted risk
- Individuals with mid‑range BMI values appear to have a higher risk of stroke than those at either very high or very low BMI levels
- Several categorical variables show weaker or context-dependent influence

### Local Explanation: Individual Prediction

The purpose of local explaination is to know why a specific patient was predicted as high or low risk. In simple words, local explaination tells us which features played an important role (for model) for a specific patient.

In [ ]:
idx = 49
idx_2 = 99

shap.plots.waterfall(shap_values_xgb[idx], show=False)
plt.title("SHAP Waterfall plot of patient 50 for XGBoost stroke prediction model")
plt.show()

In [ ]:
shap.plots.waterfall(shap_values_xgb[idx_2], show=False)
plt.title("SHAP Waterfall plot of patient 100 for XGBoost stroke prediction model")
plt.show()

In [ ]:
shap.plots.waterfall(shap_values_cat[idx], show=False)
plt.title("SHAP Waterfall plot of patient 50 for CatBoost stroke prediction model")
plt.show()

In [ ]:
shap.plots.waterfall(shap_values_cat[idx_2], show=False)
plt.title("SHAP Waterfall plot of patient 100 for CatBoost stroke prediction model")
plt.show()

The waterfall plots illustrates how individual feature contributions combine to produce the final prediction. Features pushing the prediction toward higher stroke risk are shown in red, while protective features are shown in blue. This allows for intuitive, patient-level interpretability.

### Feature Effect Analysis (Dependence Plots)

This is to analyze how feature values relate to their SHAP contributions.

#### Influence of Age

In [ ]:
shap.plots.scatter(shap_values_xgb[:, "age"], show=False)
plt.title("SHAP Scatter plot showing the influence of Age on stroke risk predictions in the XGBoost model")
plt.show()

In [ ]:
shap.plots.scatter(shap_values_cat[:, "age"], show=False)
plt.title("SHAP Scatter plot showing the influence of Age on stroke risk predictions in the CatBoost model")
plt.show()

The SHAP scatter plots shows how the SHAP value of age changes across its range. Older patients exhibit consistently higher SHAP values, confirming that age is a dominant driver of stroke risk in the model.

#### Influence of Average Glucose Level

In [ ]:
shap.plots.scatter(shap_values_xgb[:, "avg_glucose_level"], show=False)
plt.title("SHAP Scatter plot showing the influence of Average Glucose Level on stroke risk predictions in the XGBoost model")
plt.show()

In [ ]:
shap.plots.scatter(shap_values_cat[:, "avg_glucose_level"], show=False)
plt.title("SHAP Scatter plot showing the influence of Average Glucose Level on stroke risk predictions in the CatBoost model")
plt.show()

The SHAP scatter plots indicate that lower average glucose levels contribute negatively to stroke risk predictions, while higher glucose values increasingly push the model toward a higher predicted risk. This pattern is consistent across both the CatBoost and XGBoost models, suggesting that average glucose level is a strong and stable risk factor influencing stroke predictions.

#### Influence of Age with Hypertension

In [ ]:
shap.plots.scatter(shap_values_xgb[:, "age"], color=shap_values_xgb[:, "hypertension"], show=False)
plt.title("SHAP Scatter plot showing the influence of Age with Hypertension on stroke risk predictions in the XGBoost model")
plt.show()

In [ ]:
shap.plots.scatter(shap_values_cat[:, "age"], color=shap_values_cat[:, "hypertension"], show=False)
plt.title("SHAP Scatter plot showing the influence of Age with Hypertension on stroke risk predictions in the CatBoost model")
plt.show()

This visualization reveals interactions between age and hypertension, indicating that the effect of age on stroke risk is amplified for patients with hypertension.

#### Influence of Average Glucose Level with Hypertension

In [ ]:
shap.plots.scatter(shap_values_xgb[:, "avg_glucose_level"], color=shap_values_xgb[:, "hypertension"], show=False)
plt.title("SHAP Scatter plot showing the influence of Average Glucose Level with Hypertension on stroke risk predictions in the XGBoost model")
plt.show()

In [ ]:
shap.plots.scatter(shap_values_cat[:, "avg_glucose_level"], color=shap_values_cat[:, "hypertension"], show=False)
plt.title("SHAP Scatter plot showing the influence of Average Glucose Level with Hypertension on stroke risk predictions in the CatBoost model")
plt.show()

Elevated glucose levels are the dominant stroke risk predictor in the CatBoost model, with hypertension significantly amplifying this effect.

#### Analysis of Gender

In [ ]:
shap.plots.scatter(shap_values_xgb[:, "gender"], show=False)
plt.title("SHAP Scatter plot showing the influence of Gender on stroke risk predictions in the XGBoost model")
plt.show()

In [ ]:
shap.plots.scatter(shap_values_cat[:, "gender"], show=False)
plt.title("SHAP Scatter plot showing the influence of Gender on stroke risk predictions in the CatBoost model")
plt.show()

Female patients demonstrate significantly higher gender SHAP values compared to males, suggesting a strong gender bias in the model's predictions. However, this effect is an artifact of independent analysis; when other features are included, gender contributes minimally to the overall stroke risk assessment, as shown in the plot below.

In [ ]:
idx = 17

shap.plots.waterfall(shap_values_xgb[idx], show=False)
plt.title("SHAP Waterfall plot for XGBoost stroke prediction model")
plt.show()

While LIME provides local approximations of model behavior using linear surrogate models, SHAP offers explanations grounded in cooperative game theory. SHAP explanations are additive, consistent, and remain stable across different runs. In contrast to LIME, SHAP provides both global and local interpretability without requiring repeated sampling.

# LIME

In [ ]:
# %pip install lime -q
import lime
import lime.lime_tabular

# Setting up LIME
feature_names = X_train_res.columns.tolist()
class_names = ['No Stroke', 'Stroke']
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_res.values,
    feature_names=feature_names,
    class_names=class_names,
    discretize_continuous=True
)

# We'll analyze the same instances as the rest of the notebook
instance_50 = X_test.iloc[49].values
instance_100 = X_test.iloc[99].values

## Implementing LIME on RF

> ⚠️ Graphs key: Red pushes to No Stroke and Green to Stroke

In [ ]:
# Instance 50
print("Instance 50")
print("Predicted:", class_names[rf.predict(pd.DataFrame(instance_50.reshape(1, -1), columns=X_train_res.columns))[0]])
print("Actual:", class_names[y_test.iloc[49]])
exp_rf_50 = explainer.explain_instance(instance_50, rf.predict_proba, num_features=len(feature_names))
exp_rf_50.as_pyplot_figure()
plt.show()

# Instance 100
print("Instance 100")
print("Predicted:", class_names[rf.predict(pd.DataFrame(instance_100.reshape(1, -1), columns=X_train_res.columns))[0]])
print("Actual:", class_names[y_test.iloc[99]])
exp_rf_100 = explainer.explain_instance(instance_100, rf.predict_proba, num_features=len(feature_names))
exp_rf_100.as_pyplot_figure()
plt.show()

> Results

Instance 50 (prediction: No Stroke):
  - Strong protective (red): avg_glucose_level ≤ ~81.5 and bmi ≤ ~25.4 reduce risk.
  - Mild risk (green): age in ~41–59 and some work_type effect add small positive pushes.
  - Reading: Protective signals dominate overall, so the model predicts No Stroke.

Instance 100 (prediction: Stroke):
  - Dominant risk (green): age in ~59.5–74.5, work_type threshold, and higher avg_glucose_level.
  - Local counter‑effect (red): a BMI threshold can appear protective here due to local interactions; this does not imply BMI is globally protective.
  - Reading: Age and glucose are the main drivers flipping the prediction to Stroke; other features reinforce the decision.

## Implementing LIME of XGB

In [ ]:
# Instance 50
print("Instance 50")
print("Predicted:", class_names[xgb.predict(pd.DataFrame(instance_50.reshape(1, -1), columns=X_train_res.columns))[0]])
print("Actual:", class_names[y_test.iloc[49]])
exp_xgb_50 = explainer.explain_instance(instance_50, xgb.predict_proba, num_features=len(feature_names))
exp_xgb_50.as_pyplot_figure()
plt.show()

# Instance 100
print("Instance 100")
print("Predicted:", class_names[xgb.predict(pd.DataFrame(instance_100.reshape(1, -1), columns=X_train_res.columns))[0]])
print("Actual:", class_names[y_test.iloc[99]])
exp_xgb_100 = explainer.explain_instance(instance_100, xgb.predict_proba, num_features=len(feature_names))
exp_xgb_100.as_pyplot_figure()
plt.show()

> Results

Instance 50 (predicted: No Stroke)

- avg_glucose_level ≤ ~81.5 is strongly protective.
- bmi ≤ ~25.4 also reduces risk.
- age in ~41–59 adds some local risk.
- work_type threshold contributes a smaller positive push.
Takeaway: Low glucose and normal BMI dominate, outweighing moderate age/work-related risk, so the model stays with No Stroke.

Instance 100 (predicted: Stroke)
- age in ~59.5–74.5 is the largest driver toward Stroke.
- work_type threshold contributes meaningful positive risk.
- avg_glucose_level > ~170 adds further risk.

> Post Analysis Note:

bmi > ~31.8 shows a negative (red) contribution here; that can happen locally due to feature interactions and how LIME bins features. It doesn’t mean high BMI is globally protective, more so just that, for this patient’s mix of values, BMI is not the main driver of the Stroke prediction.

Takeaway: Age and high glucose chiefly flip the prediction to Stroke; other features reinforce the direction.

- Practical note:
  - Prefer medically plausible patterns (older age, higher glucose increasing risk). If LIME highlights implausible drivers, revisit preprocessing (e.g., outlier handling) or constraints.

- Reminder:
  - LIME is local to the chosen instance and explains “why this prediction here,” not global importance.

## Interactive Widget for all Instances

In [ ]:
from ipywidgets import IntSlider, Dropdown, HBox, VBox, Output
from IPython.display import display

try:
    viewer_container.close()
except Exception:
    pass

# Controls
idx_slider = IntSlider(value=0, min=0, max=len(X_test)-1, step=1, description='Instance')
model_picker = Dropdown(options=[('Random Forest', 'rf'), ('XGBoost', 'xgb')], value='rf', description='Model')

# Single output area
out = Output()

# Render once per change into the single output area
def render(change=None):
    with out:
        out.clear_output(wait=True)
        idx = idx_slider.value
        model_key = model_picker.value
        model = rf if model_key == 'rf' else xgb
        x = X_test.iloc[idx].values

        # Predicted/Actual labels
        pred_label = model.predict(pd.DataFrame(x.reshape(1, -1), columns=X_train_res.columns))[0]
        actual_label = y_test.iloc[idx]
        print(f"Instance {idx}")
        print("Predicted:", class_names[pred_label])
        print("Actual:", class_names[actual_label])

        # Instance values
        try:
            decoded = transformCategory(X_test.iloc[[idx]])
            print("\nInstance values:")
            display(decoded)
        except Exception:
            pass

        # LIME explanation
        exp = explainer.explain_instance(
            x,
            lambda d: model.predict_proba(pd.DataFrame(d, columns=feature_names)),
            num_features=len(feature_names)
        )
        exp.as_pyplot_figure()
        plt.show()

        print("\nVaraibles and their influence:")
        for rule, weight in exp.as_list():
            print(f"- {rule} → {weight:.3f}")

# Hook up observers (only once, for this container)
idx_slider.observe(render, names='value')
model_picker.observe(render, names='value')

# Display container and do initial render
viewer_container = VBox([HBox([idx_slider, model_picker]), out])
display(viewer_container)
render()


## SP-LIME

Up until now, our analysis has been more local and not necessarily representative of the overall model. In the following section, with a motivation to garner a more global analysis, we will implement SP-LIME, "a method that selects a set of representative instances with explanations to address the “trusting the model problem, via submodular optimization." [1]

> [1] Quote from Ribeiro, M. T., Singh, S., & Guestrin, C. (2016). "Why Should I Trust You?": Explaining the Predictions of Any Classifier. ArXiv. https://arxiv.org/abs/1602.04938

In [ ]:
# SP-LIME Implementation, submodular pick for global and representative explanations for RF & XGB
N = len(X_test)
D = X_test.iloc[:N].values

# RF
sp_rf = SubmodularPick(
    explainer,
    D,
    lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
    num_features=10,
    num_exps_desired=8
)

# XGB
sp_xgb = SubmodularPick(
    explainer,
    D,
    lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
    num_features=10,
    num_exps_desired=8
)

In [ ]:
# Displaying the SP-LIME representative instances

rf_positions = list(getattr(sp_rf, 'V', []))
xgb_positions = list(getattr(sp_xgb, 'V', []))

print("SP-LIME RF representative instances:")
print(f"Positions in X_test: {rf_positions}")
display(X_test.iloc[rf_positions].reset_index(drop=True))

print("SP-LIME XGB representative instances:")
print(f"Positions in X_test: {xgb_positions}")
display(X_test.iloc[xgb_positions].reset_index(drop=True))

> Insights

Submodular Pick (SP-LIME) selected a small, representative set of instances from `X_test` that best covers the diversity of local LIME explanations. In our case, we have generated 8 such instances, displayed above. The motivation stems from the fact that instance-by-instance LIME, what we did in the last section, is local. Here, SP-LIME lifts it to a global view without scanning the entire dataset.

> Conclusions:

- Across both RF and XGB, age and avg_glucose_level consistently appear as top drivers, with bmi and work_type also influential. 
- Cardiovascular indicators (hypertension, heart_disease) contribute but with smaller average weights than age/glucose. 
- The representative subset reflects diverse categorical combinations (gender, residence, marriage), which helps capture different decision regions. Decoding confirms categories are sensible and interpretable (no unexpected codes). This improves the trust and use of instance level reviews for clinicians and stakeholders.


> What the results show:

- Consistent global drivers across models:
  - Age is the strongest contributor in both RF and XGB.
  - Avg glucose level and BMI are also influential, with work_type showing moderate importance.
- Additional, smaller contributors:
  - Residence_type and smoking_status have non-trivial but smaller average impacts.
  - Binary comorbids (hypertension, heart_disease) contribute less on average globally, but can be decisive for specific individuals.
- Local variability:
  - Across the representative set, feature importance patterns vary by instance, reinforcing that LIME explanations are local.
  - Previously observed LIME fidelity scores (exp.score) and distributions indicate decent but instance-dependent surrogate fit.


Below we will look at two metrics that help us understand the LIME analysis in a more quantitative way

## LIME Metrics

### LIME Weights

In [ ]:
# Average absolute LIME weight for RF
N = len(X_test)
weights = pd.Series(0.0, index=feature_names)
counts = pd.Series(0, index=feature_names)

for idx in range(N):
    x = X_test.iloc[idx].values
    exp = explainer.explain_instance(
        x,
        lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    for rule, w in exp.as_list():
        fname = rule.split(' ')[0]
        if fname in weights.index:
            weights[fname] += abs(w)
            counts[fname] += 1

avg_abs = (weights / counts.replace(0, np.nan)).dropna().sort_values(ascending=False).head(10)
plt.figure(figsize=(8, 4))
avg_abs.plot(kind='barh', color='teal')
plt.gca().invert_yaxis()
plt.title('RF: Avg absolute LIME weight')
plt.xlabel('|weight|')
plt.tight_layout()
plt.show()

In [ ]:
# Average absolute LIME weight per feature for XGB

N = len(X_test)
weights = pd.Series(0.0, index=feature_names)
counts = pd.Series(0, index=feature_names)

for idx in range(N):
    x = X_test.iloc[idx].values
    exp = explainer.explain_instance(
        x,
        lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    for rule, w in exp.as_list():
        fname = rule.split(' ')[0]
        if fname in weights.index:
            weights[fname] += abs(w)
            counts[fname] += 1

avg_abs = (weights / counts.replace(0, np.nan)).dropna().sort_values(ascending=False).head(10)
plt.figure(figsize=(8, 4))
avg_abs.plot(kind='barh', color='purple')
plt.gca().invert_yaxis()
plt.title('XGB: Avg absolute LIME weight')
plt.xlabel('|weight|')
plt.tight_layout()
plt.show()

Average absolute LIME weights tells us how strongly a feature influences the model's prediction across many local LIME explanations. We take the absolute value of the weight bec in this case we are more interested in the influence, regardless of whether it is positive or negative. This allows us to then average values across many instances and understand which features are typically most impactful locally.

> Insights

In both models, the top two features with the most impact are the the same, namely, age and average glucose level. Comparing this with scientific literature, this checks out. These results go a long way into building trust on the model. 

However, at the same time, something suspicious is that prior hypertension and heart disease existence has almost no influence (<0.1) on the prediction. As a reminder, we remember that these magnitudes are local, and they depend on LIME binning. They should be used together with fidelity (see below)

### LIME Fidelity

In [ ]:
# LIME fidelity distribution (RF vs XGB) with counts

N = min(300, len(X_test))
rf_scores, xgb_scores = [], []

for idx in range(N):
    x = X_test.iloc[idx].values
    exp_rf = explainer.explain_instance(
        x,
        lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    rf_scores.append(getattr(exp_rf, 'score', np.nan))

    exp_xgb = explainer.explain_instance(
        x,
        lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    xgb_scores.append(getattr(exp_xgb, 'score', np.nan))

plt.figure(figsize=(8, 4))
counts_list, bins, containers = plt.hist([rf_scores, xgb_scores], bins=20, label=['RF', 'XGB'], alpha=0.7)
plt.title('LIME fidelity (local surrogate R^2) distribution')
plt.xlabel('R^2 (score)')
plt.ylabel('Count')
plt.legend()

# Annotation of counts
for counts, container in zip(counts_list, containers):
    for c, rect in zip(counts, container.patches):
        if c > 0:
            x = rect.get_x() + rect.get_width() / 2
            y = rect.get_height()
            plt.text(x, y + max(0.5, y * 0.02), str(int(round(c))), ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

This shows us the distribution of the local surrogate R² (fidelity) for RF and XGB. For the right cluster i.e. R^2 of ~0.5-0.7 or so, both models perform fairly well. In this cluster, we can trust both the direction and the magnitude of the feature weights.

For the left cluster, we have low fidelity scores. It tells us that the local linear surrogate struggles to explain the predictions. We can only use LIME for a rough directional hint here. 

In [ ]:
# Predicted probability vs LIME fidelity (RF and XGB)

N = min(300, len(X_test))
rf_probs, xgb_probs = [], []
rf_scores, xgb_scores = [], []

for idx in range(N):
    x = X_test.iloc[idx].values
    df = pd.DataFrame(x.reshape(1, -1), columns=feature_names)

    # model probabilities
    rf_probs.append(rf.predict_proba(df)[0, 1])
    xgb_probs.append(xgb.predict_proba(df)[0, 1])

    # LIME fidelity (local surrogate R^2)
    exp_rf = explainer.explain_instance(
        x,
        lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    rf_scores.append(getattr(exp_rf, 'score', np.nan))

    exp_xgb = explainer.explain_instance(
        x,
        lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    xgb_scores.append(getattr(exp_xgb, 'score', np.nan))

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
axes[0].scatter(rf_probs, rf_scores, alpha=0.6, color='skyblue', s=18)
axes[0].set_title('RF: prob vs LIME fidelity')
axes[0].set_xlabel('Predicted stroke probability')
axes[0].set_ylabel('LIME fidelity (R^2)')

axes[1].scatter(xgb_probs, xgb_scores, alpha=0.6, color='orange', s=18)
axes[1].set_title('XGB: prob vs LIME fidelity')
axes[1].set_xlabel('Predicted stroke probability')

plt.tight_layout()
plt.show()

> Predicted probability vs LIME fidelity — insights

- RF:
    - We see two clusterings, a higher‑fidelity band around ~0.55–0.65 (mostly at lower risk) and a lower band around ~0.1–0.25 across the rest.
    - This suggests RF’s decisions are easier to approximate locally for many low‑risk predictions, while mid‑to‑high risk cases often look more complex (lower R²).
- XGB:
    - More spread and generally lower fidelity on average than RF, with many points clustered at low R² (≈0.05–0.2) across the full probability range.
    - This hints XGB’s decision boundary is more nonlinear/complex, making a simple local surrogate (like LIME’s linear model) fit less tightly.

> Fidelity:

Fidelity is a trust measure for local explanations. It tells us how well the LIME surrogate model represents the original model's outputs on the samples. A high fidelity i.e. close to 1 tells us that the surrogate matches the original model fairly well and vice versa. 

With a high fidelity i.e. >0.6, we can trust both the direction and the relative magnitude of the LIME weights. A moderate fidelity i.e. 0.3-0.6, we should treat LIME as a mere directional insight i.e. what features push the predictions up/down. As for low fidelity i.e. <0.3 we should realize that the local linear surrogate does not do a very good job of fitting to the model. 

In this case, in the cluster of High R², we can treat the LIME weights as a more faithful picture of the model’s local behavior. Meanwhile for Low R², we use LIME weights for rough influence measure i.e. which features push up or down risk. Again, this is not a precise magnitude, thus results should be cross checked with a professional for instance.

> Conclusions:

We can see that on average that our LIME implementation does a better job at approximating to the RF compared to XGB. 
What next? this can be studied and compared with other datasets to see whether LIME works better with RF as compared to XGB. If this is indeed the case then this will help us contribute towards the open field of xAI research.
Moreover, after this visualization, we can tell the doctors which model to trust more depending on whether they want to prioritize recall or precision in their predictions.

## Stats of all Instances

In [ ]:
N = len(X_test)

rf_probs, xgb_probs = [], []
rf_scores, xgb_scores = [], []

for idx in range(N):
    x = X_test.iloc[idx].values
    df = pd.DataFrame(x.reshape(1, -1), columns=feature_names)
    
    # Original model probabilities
    rf_probs.append(rf.predict_proba(df)[0, 1])
    xgb_probs.append(xgb.predict_proba(df)[0, 1])
    
    # Comparing local surrogate fidelity with the model
    exp_rf = explainer.explain_instance(
        x, 
        lambda d: rf.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=5  # fewer features for speed
    )
    rf_scores.append(getattr(exp_rf, 'score', np.nan))
    
    exp_xgb = explainer.explain_instance(
        x, 
        lambda d: xgb.predict_proba(pd.DataFrame(d, columns=feature_names)),
        num_features=10
    )
    xgb_scores.append(getattr(exp_xgb, 'score', np.nan))

summary = pd.DataFrame({
    'model': ['Random Forest', 'XGBoost'],
    'proba_mean': [np.mean(rf_probs), np.mean(xgb_probs)],
    'proba_median': [np.median(rf_probs), np.median(xgb_probs)],
    'proba_std': [np.std(rf_probs), np.std(xgb_probs)],
    'lime_fidelity_mean': [np.nanmean(rf_scores), np.nanmean(xgb_scores)],
    'lime_fidelity_median': [np.nanmedian(rf_scores), np.nanmedian(xgb_scores)]
})

print(f"Sample size: {N}")
display(summary)

> What the cell computes

- For each test instance, it collects:
  - Predicted stroke probabilities from RF and XGB (rf_probs, xgb_probs).
  - LIME surrogate fidelity scores for RF and XGB (rf_scores, xgb_scores), i.e., how well the local linear model approximates the black-box around that instance.

> Interpreting the results

- Stroke probability (original model outputs)
  - XGBoost shows higher mean/median stroke probability than Random Forest. This essentailly tells us XGBoost is more risk‑sensitive and will flag more patients as likely stroke. This is good! Recall over Precision is best in medical fields
  - XGBoost’s probabilities are also more spread out (higher variability), while Random Forest looks more conservative/stable. Expect XGBoost to boost recall but potentially lower precision.

- LIME fidelity (how well the local surrogate mimics the model)
  - Average fidelity is modest (roughly in the 0.27–0.33 range, while the median is lower). Normal for fast, lightweight settings and reminds us LIME is an approximation.
  - Use LIME primarily for the direction and main drivers (which features push toward Stroke vs No Stroke). Avoid over‑interpreting tiny weight differences.
  - We are aware that fidelity can be improved by adjusting kernel width, or allowing more features in the local surrogate, but as far as analysis has to go, this suffices + kept it small for speed

- The mean/median of rf_probs vs xgb_probs show how conservative or aggressive each model is. Higher averages imply the model is assigning more cases near the decision threshold.
- LIME fidelity (exp.score):
  - Higher average fidelity implies explanations more faithfully capture local behavior. Low-fidelity instances deserve caution; the local rules may be unstable or sensitive to perturbations.
  - In our case, RF has a higher fidelity score implying that its local explanations are more reliable for decision support.

> How to use this?
- We should prefer the model that achieves acceptable probability calibration and higher average LIME fidelity in cohorts of interest. If RF is more faithful locally for older patients, doctors should use RF-driven explanations for that subgroup.
- Having low fidelity does not mean we should discard the LIME aproximation altogether; rather we should flag low fidelity instances for additional review. For these cases, we should avoid relying solely on local rules; consider global checks or alternative explanation methods (SHAP etc.)

## Conclusions with Cohort-Wise Stats

In [ ]:
# Visualize cohort-wise stats (simple bar charts)

if 'results' not in globals() or not isinstance(results, dict) or len(results) == 0:
    print("[warn] No cohort results found. Run the cohort-wise stats cell first.")
else:
    for cohort_key, dfc in results.items():
        if not isinstance(dfc, pd.DataFrame) or dfc.empty:
            print(f"[info] Empty results for cohort '{cohort_key}', skipping.")
            continue

        dfc = dfc.sort_index()

        # Observed vs predicted mean probabilities
        comp = pd.DataFrame({
            'Observed rate': dfc['pos_rate'],
            'RF mean prob': dfc['rf_prob_mean'],
            'XGB mean prob': dfc['xgb_prob_mean'],
        })
        ax = comp.plot(kind='bar', figsize=(8, 4))
        ax.set_title(f"{cohort_key}: observed vs predicted mean probability")
        ax.set_ylabel("Probability")
        plt.xticks(rotation=0)
        plt.legend(loc='best')
        plt.tight_layout()
        plt.show()

        # Average LIME fidelity
        fid = pd.DataFrame({
            'RF fidelity': dfc['rf_fidelity_mean'],
            'XGB fidelity': dfc['xgb_fidelity_mean'],
        })
        ax = fid.plot(kind='bar', figsize=(8, 4))
        ax.set_title(f"{cohort_key}: average LIME fidelity")
        ax.set_ylabel("LIME exp.score")
        plt.xticks(rotation=0)
        plt.legend(loc='best')
        plt.tight_layout()
        plt.show()

        # Calibration gap bars (mean prob - observed)
        gap = pd.DataFrame({
            'RF gap': dfc['rf_calibration_gap'],
            'XGB gap': dfc['xgb_calibration_gap'],
        })
        ax = gap.plot(kind='bar', figsize=(8, 3))
        ax.axhline(0, color='black', linewidth=0.8)
        ax.set_title(f"{cohort_key}: calibration gap (mean prob - observed)")
        ax.set_ylabel("Gap")
        plt.xticks(rotation=0)
        plt.legend(loc='best')
        plt.tight_layout()
        plt.show()

### Cohort-wise Statistics Summary and Analysis

> Motivation

Stroke risk is not uniform across populations. Age, cardiovascular health, hypertension, and lifestyle factors (smoking) are well-established risk modifiers. With the above visualizations, our goal is to identify whether our models (Random Forest and XGBoost) and LIME explanations behave consistently across different demographic and clinical groups, or if certain cohorts show:
- Calibration problems: models over/underestimate risk for specific groups.
- Fidelity issues: LIME explanations are less trustworthy (less faithful) in certain cohorts.
- Potential fairness concerns: systematic biases in predictions or unreliable explanations for vulnerable groups.

> 1. Observed vs Predicted Mean Probability Chart
- X-axis: cohort groups (e.g., age bands <40, 40-60, >60).
- Y-axis: probability (0–1).
- Three bars per group: orange=observed rate, blue=RF mean probability, green=XGB mean probability.

Interpretation:
- If all three bars align closely, the model is well-calibrated for that cohort.
- If RF or XGB bars are consistently above the orange bar, the model is overestimating risk (too aggressive) for that group.
- If bars are consistently below, the model is underestimating risk (too conservative) for that group.
- Look for patterns: do older patients or those with hypertension see larger discrepancies?

Action: Cohorts with large gaps can improve with threshold adjustment or model retraining on that subpopulation.



> 2. Average LIME Fidelity Chart
- X-axis: cohort groups.
- Y-axis: LIME exp.score (0–1, where 1 is perfect faithfulness).
- Two bars per group: blue=RF fidelity, orange=XGB fidelity.

Interpretation:
- Fidelity < 0.5: local explanations are unreliable; the linear surrogate model does not capture the decision boundary well.
- Fidelity 0.5–0.8: moderate reliability; use explanations as a starting point but verify with domain knowledge.
- Fidelity > 0.8: high confidence; explanations are trustworthy for decision support.

Common patterns:
- Cohorts with extreme probabilities (very high or very low risk) typically have higher fidelity because the decision is "clearer." This can help us pin point features that contribute the most to the prediction.
- Cohorts with mid-range probabilities (borderline cases) often show lower fidelity because small perturbations can flip the prediction.
- If one model (in our case, RF) has consistently higher fidelity, use RF-driven explanations as primary and XGB as secondary.

Action: Flag cohorts with low fidelity (<0.5) for manual review. For borderline cases in those cohorts, consider alternative explainability methods.



> 3. Calibration Gap Chart
- X-axis: cohort groups.
- Y-axis: gap = mean predicted probability − observed rate.
- Two bars per group: blue=RF gap, orange=XGB gap.

Interpretation:
- Bars above zero: overestimation (model is too confident).
- Bars below zero: underestimation (model is too conservative).
- Magnitude indicates how far off the model is. A gap of ±0.1 means the model is off by ~10 percentage points.

Examples:
- RF for age >60 showing gap of around +0.50 means RF predicts ~35% stroke risk but far less of them will actually have strokes in that group. This is problematic as patients may receive unnecessary interventions.
- XGB for smokers showing gap of around 0.3 means XGB predicts ~30% stroke risks but less actually have strokes.

Action:
- We could use these calibration plots to select thresholds that match clinical cost-benefit trade-offs.
- Report gaps to doctors so they understand model limitations per demographic.

# Counterfactual Explanations

## A glance over Dice 
DiCE is a library developed by Microsoft that specializes in generating a set of diverse counterfactual examples. The "diversity" aspect is crucial because there is rarely just one way to change an outcome. DiCE attempts to find multiple paths to the desired prediction (e.g., changing BMI vs. changing smoking status) while ensuring that the suggested changes are:

1/Diversity:

    - DiCE doesn’t just generate a single counterfactual; it generates a set of alternatives.
    Example: To get a loan approved, one option might be “increase income,” another might be “reduce debt,” and yet another could be “improve credit score.”

-->This helps users choose a path that is practical for them.

2/Proximity

    - DiCE tries to make counterfactuals close to the original input, so the suggested changes are minimal and realistic.
    Example: If your BMI is 30, suggesting a BMI of 18 might technically change the outcome but is too drastic. DiCE prefers smaller, feasible changes.

--> Ensure reasonable values

3/Feasibility

    - DiCE ensures counterfactuals make sense according to the data.
    For example, it won’t suggest “age = -5” or “married = yes AND single = yes.”

-->This is done by respecting constraints and distributions in the training data.

4/Flexibility

    - DiCE can be used with any predictive model (tree-based, neural networks, etc.) because it works with the model’s predictions, not its internal workings.
    - Handles **continuous, categorical, and ordinal features**.  


Note: DiCE works best with **scikit-learn** and **TensorFlow**. Other frameworks may require additional adjustments to work correctly.



## Explainable Insights

In [ ]:
categorical_maps = {
    'gender': {0: 'Female', 1: 'Male', 2: 'Other'},
    'ever_married': {0: 'No', 1: 'Yes'},
    'work_type': {0: 'Govt_job', 1: 'Never_worked', 2: 'Private', 3: 'Self-employed', 4: 'children'},
    'Residence_type': {0: 'Rural', 1: 'Urban'},
    'smoking_status': {0: 'Unknown', 1: 'formerly smoked', 2: 'never smoked', 3: 'smokes'}
}

In [ ]:
## Map the catetory to human-readable
def transformCategory(df):
    df_copy = df.copy()  
    for col, mapping in categorical_maps.items():
        if col in df_copy.columns:  
            df_copy[col] = df_copy[col].map(mapping)
    return df_copy

### Generating CEs using DiCE

### For Random Forest

In [ ]:
# %pip install dice-ml -q
import dice_ml

categorical_features = ['gender', 'hypertension', 'heart_disease',
                        'ever_married', 'work_type', 'Residence_type', 'smoking_status']

continuous_features = ['age', 'avg_glucose_level', 'bmi']

d = dice_ml.Data(
    dataframe=X_train_res.assign(target=y_train_res), 
    continuous_features=continuous_features,
    outcome_name='target'
)

m = dice_ml.Model(model=rf, backend='sklearn')  
exp = dice_ml.Dice(d, m, method='random')  # method to generate CFs


query_instance = X_test.loc[[4858]].copy() # Can change this to different instance
   

cf = exp.generate_counterfactuals(
    query_instance,
    total_CFs=3,
    desired_class="opposite",
)

cf_df = cf.cf_examples_list[0].final_cfs_df  

transformCategory(cf_df)

In [ ]:
predicted_target = rf.predict(query_instance)  
query_instance.loc[:, "target"] = predicted_target
transformCategory(query_instance)

### So this works totally fine because DiCE supports directly sklearn and TF. 

### For XGBoost

In [ ]:
# %pip install dice-ml -q
import dice_ml

categorical_features = ['gender', 'hypertension', 'heart_disease',
                        'ever_married', 'work_type', 'Residence_type', 'smoking_status']

continuous_features = ['age', 'avg_glucose_level', 'bmi']

d = dice_ml.Data(
    dataframe=X_train_res.assign(target=y_train_res), 
    continuous_features=continuous_features+categorical_features,
    outcome_name='target'
)

m = dice_ml.Model(model=xgb, backend='sklearn')  
exp = dice_ml.Dice(d, m, method='random')  


query_instance = X_test.loc[[196]].copy()
   

cf = exp.generate_counterfactuals(
    query_instance,
    total_CFs=3,
    desired_class="opposite",
)

cf_df = cf.cf_examples_list[0].final_cfs_df  

transformCategory(cf_df)

In [ ]:
predicted_target = xgb.predict(query_instance)  
query_instance.loc[:, "target"] = predicted_target
transformCategory(query_instance)

### Identified Issue and Resolution

During the analysis, an unexpected issue was encountered due to a mismatch between DiCE and XGBoost. Although all feature columns were correctly defined with numerical data types (int and float) in the original dataset, DiCE internally converts some features to object type during counterfactual generation. This behavior leads to a conflict, as XGBoost does not accept object-typed inputs, resulting in runtime errors.

After investigation, two possible solutions were identified:

1/ Using the kdtree method
This approach works reliably because it generates counterfactuals by copying existing instances from the dataset rather than perturbing feature values. As a result, the original data types are preserved, preventing any type mismatch between DiCE and XGBoost. This method fully resolves the issue without side effects.

2/ Treating categorical features as continuous features
An alternative (though less principled) solution is to include all categorical features in the continuous_features list and treat them as numerical values. While this approach technically resolves the data type conflict and works well with most DiCE methods, it must be used with caution. In particular, when using the genetic algorithm or Random , this setup may produce unrealistic or invalid categorical values due to numerical perturbations. However, Random method, this issue was not observed, and the categorical values remained valid.

### For CatBoost

In [ ]:
# %pip install dice-ml -q
import dice_ml

categorical_features = ['gender', 'hypertension', 'heart_disease',
                        'ever_married', 'work_type', 'Residence_type', 'smoking_status']

continuous_features = ['age', 'avg_glucose_level', 'bmi']

d = dice_ml.Data(
    dataframe=X_train_res.assign(target=y_train_res), 
    continuous_features=continuous_features+categorical_features,
    outcome_name='target'
)

m = dice_ml.Model(model=catboost_model, backend='sklearn')  
exp = dice_ml.Dice(d, m, method='kdtree')  


query_instance = X_test.iloc[99:100].copy() # Can change this 
   

cf = exp.generate_counterfactuals(
    query_instance,
    total_CFs=3,
    desired_class="opposite",
)

cf_df = cf.cf_examples_list[0].final_cfs_df  

transformCategory(cf_df)

In [ ]:
predicted_target = catboost_model.predict(query_instance)  
query_instance.loc[:, "target"] = predicted_target
transformCategory(query_instance)

### Identified Issue and Resolution

A similar issue was observed when using CatBoost with DiCE. During counterfactual generation, DiCE attempts to transform integer categorical values (e.g., 1 of type int) into floating-point values (e.g., 1.0 of type float). This behavior is not permitted by CatBoost, which enforces strict type consistency for categorical features.

If categorical features are not included in continuous_features, DiCE automatically converts integer-encoded categories (e.g., 0, 1) into pandas category dtype. However, this also leads to an error in CatBoost, as CatBoost requires all categorical columns to be explicitly listed in cat_features. This results in the following error:

"CatBoostError: features data: pandas.DataFrame column 'hypertension' has dtype 'category' but is not in cat_features list"

Due to these constraints, the only reliable solution is to include categorical features in continuous_features and use the kd_tree method. This method generates counterfactuals by copying existing instances rather than numerically perturbing feature values, thereby preserving valid feature values and avoiding data type conflicts with CatBoost.

### In general, DiCE provides robust and reliable support primarily for models built with scikit-learn and TensorFlow. When used with other frameworks such as XGBoost and CatBoost, additional compatibility issues may arise—particularly related to data type handling for categorical features—often requiring workarounds or constrained generation methods (e.g., kd_tree) to ensure correct functionality.

## Counterfactual Analysis

## CEs for Random Forest

### 1/

Original patient profile:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 43  | No           | No            | Yes     | Private   | Urban     | 86.67       | 33.3 | never smoked   | 0      |

Counterfactuals generated:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 43  | 0            | 0             | Yes     | Private   | Urban     | 110.87      | 33.3 | formerly smoked | 1      |
| Female | 43  | 0            | 0             | Yes     | Private   | Urban     | 69.48       | 33.3 | never smoked   | 1      |
| Female | 43  | 0            | 0             | Yes     | Private   | Urban     | 60.55       | 33.3 | never smoked   | 1      |

### Interpretation

1. **Target change:** All counterfactuals flip the predicted stroke from 0 → 1.  
2. **Feature changes:** Only the **average glucose level** and **smoking status** vary significantly.  
3. **Unrealistic pattern:**  
   - Two counterfactuals suggest that **lowering average glucose level (69.48, 60.55)** increases stroke risk.  
   - This is **not clinically plausible**, as lower glucose does not increase stroke probability.  

### Actionable insight

- This highlights that the model may have learned **spurious relationships** from rare cases.  
- To generate **meaningful and medically realistic counterfactuals and Model** , extreme or low-probability feature values should be filtered out during analysis.



In [ ]:
print(data['stroke'][(data["avg_glucose_level"]<=70) & (data['stroke'] == 1)].count() / data['stroke'][(data["avg_glucose_level"]<=70) & (data['stroke'] == 0)].count() * 100)

> Go deeper we can see that the case that the patient has **avg_glucose_level <= 70** and has stroke is only **3,7%**. So for more reasonable analysis, we should consider those as outliers

### 2/

Original patient profile:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 17  | No           | No            | No      | Govt_job  | Urban     | 68.91       | 23.0 | Unknown        | 0      |

Counterfactuals generated:

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 76.6 | 0            | 0             | No      | Govt_job  | Urban     | 222.16      | 23.0 | Unknown        | 1      |
| Male   | 57.1 | 0            | 0             | No      | Govt_job  | Urban     | 179.49      | 23.0 | Unknown        | 1      |
| Male   | 65.8 | 0            | 0             | No      | Govt_job  | Urban     | 227.21      | 23.0 | Unknown        | 1      |

### Interpretation

1. **Target change:** All counterfactuals flip the predicted stroke from 0 → 1.  
2. **Feature changes:** The primary changes are in **age** and **average glucose level**, which increase in the CFs.  
3. **Reasonable pattern:**  
   - Older age and higher glucose levels are associated with increased stroke risk.  
   - This aligns with clinical expectations and literature.  

### Actionable insight

- By removing outliers from the dataset, the counterfactuals now reflect **plausible and medically meaningful scenarios**.  
- This demonstrates that careful preprocessing (outlier removal) is essential for **trustworthy counterfactual explanations** in healthcare models.


### 3/

Original patient profile:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 5   | No           | No            | No      | children  | Rural     | 102.04      | 18.5 | Unknown        | 0      |

Counterfactuals generated:

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type   | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|------------|-----------|-------------|------|----------------|--------|
| Female | 49.9 | 0            | 0             | No      | Private    | Rural     | 102.04      | 18.5 | Unknown        | 1      |
| Female | 74.4 | 0            | 0             | No      | Never_worked | Rural   | 102.04      | 18.5 | Unknown        | 1      |
| Female | 76.3 | 0            | 0             | No      | Govt_job   | Rural     | 102.04      | 18.5 | Unknown        | 1      |

### Interpretation

1. **Target change:** All counterfactuals flip stroke prediction from 0 → 1.  
2. **Feature changes:** The main changes are **age** and **work type**, while glucose and BMI remain constant.  
3. **Insight:**  
   - Even at low BMI and normal glucose levels, older age and occupational stress factors (work type) increase predicted stroke risk.  
   - This suggests that **stress-related factors might be important contributors** to stroke risk in the model.  

### Actionable insight

- Counterfactual analysis highlights that **non-traditional risk factors** such as age-related stress and occupational exposure can strongly influence predicted stroke risk.  
- Proper handling of stress and lifestyle factors is crucial, especially for individuals in high-risk age groups.


### 4/

**Original patient profile:**  

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 61.0 | 1            | 1             | Yes     | Govt_job  | Rural     | 86.06       | 34.8 | never smoked   | 1      |

**Counterfactuals generated:**  

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI   | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|-------|----------------|--------|
| Male   | 61.0 | 1            | 1             | Yes     | children  | Rural     | 86.06       | 96.1  | never smoked   | 0      |
| Male   | 32.9 | 1            | 1             | Yes     | Govt_job  | Rural     | 86.06       | 34.8  | never smoked   | 0      |
| Male   | 61.0 | 1            | 1             | Yes     | Govt_job  | Rural     | 67.00       | 34.8  | never smoked   | 0      |

### Interpretation

1. **Target change:** All counterfactuals flip stroke prediction from 1 → 0.  
2. **Feature changes and plausibility:**  
   - The most notable change is **BMI = 96.1** in one counterfactual, which is **clinically implausible**.  
3. **Insight:**  
   DiCE produces implausible clinical cases, e.g., suggesting that higher BMI lowers stroke risk, which contradicts real-world evidence.
   
### Actionable insight

- Counterfactual analysis reveals that **extreme BMI changes dominate the model’s predictions**, but these are **not realistic**.  
- For meaningful interpretation, **filtering out implausible counterfactuals** is necessary.  



In [ ]:
data[(data['bmi']>=50) & (data['stroke']==1)]

In [ ]:
data[(data['bmi']>=50) & (data['stroke']==0)]

#### Upon examining the dataset and generated counterfactuals, we observed several instances with **extremely high BMI values (≥ 50)** that produce unrealistic or implausible predictions. Due to the dataset's high class imbalance, although higher BMI should generally increase stroke risk, the majority of these high-BMI instances have **stroke = 0**, which skews the model’s behavior. These instances can be considered **outliers**, and since they represent a small portion of the data, it is reasonable to remove them to improve the plausibility and reliability of subsequent analyses.


### 5/

**Original patient profile:**  

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 45.0 | 0            | 0             | Yes     | Govt_job  | Urban     | 55.47       | 19.8 | smokes         | 0      |

**Counterfactuals generated:**  

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 51.9 | 0            | 0             | Yes     | Private   | Urban     | 71.40       | 44.3 | smokes         | 1      |
| Female | 80.8 | 0            | 0             | No      | Private   | Urban     | 71.40       | 28.4 | smokes         | 1      |
| Female | 66.8 | 0            | 0             | Yes     | Private   | Urban     | 158.47      | 28.4 | smokes         | 1      |

### Interpretation

1. **Target change:** All counterfactuals flip stroke prediction from 0 → 1.  
2. **Feature changes and plausibility:**  
   - Gender changed from Male → Female in all counterfactuals.  
   - Age varies widely (45 → 51.9, 80.8, 66.8).  
   - Work type changed from Govt_job → Private.  
   - BMI is extreme in one counterfactual (44.3) and normal in others.  
   - Avg glucose increased substantially in one counterfactual (55.47 → 158.47).  
   - Some changes (like large BMI jump or very high glucose) may be **clinically unrealistic**.  
3. **Insight:**  
   - The model is **very sensitive to gender**, which can flip the predicted risk.  

### Actionable insight

- Counterfactual analysis shows that **changing gender, glucose, or BMI** can alter predictions.  
- For meaningful interpretation, **focus on plausible counterfactuals** within realistic clinical ranges.  


## CEs for XGBoost

### 1/

**Original patient profile:**  

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 45.0 | No           | No            | Yes     | Govt_job  | Urban     | 55.47       | 19.8 | smokes         | 0      |

**Counterfactuals generated:**  

| Gender | Age   | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 60.2  | 0            | 0             | Yes     | Govt_job  | Urban     | 195.75      | 19.8 | smokes         | 1      |
| Male   | 81.4  | 1            | 0             | Yes     | Govt_job  | Urban     | 199.75      | 19.8 | smokes         | 1      |
| Female | 82.0  | 0            | 0             | Yes     | Govt_job  | Urban     | 152.70      | 19.8 | smokes         | 1      |

### Interpretation

1. **Target change:** All counterfactuals flip the predicted stroke from 0 → 1.  

2. **Feature changes:**  
   - **Age** increases substantially in all counterfactuals.  
   - **Hypertension** changes in one counterfactual.  
   - **Gender** changes in one counterfactual.  
   - **Average glucose level** rises sharply in all counterfactuals.  
   - Other features (BMI, marital status, work type, residence, smoking) remain unchanged.  

3. **Unrealistic pattern:**  
   - The counterfactual suggesting **gender change (Male → Female)** may not be practically actionable.  But still the somehow the Female has more chance to have a stroke than Male
   - At first glance, extremely high **average glucose levels (152.7–199.75)** are likely **clinically extreme** for someone with BMI 19.8 and age 45–82.  But, while we often associate high glucose (blood sugar) with higher body weight, it is a common misconception that a low or normal BMI "protects" you from metabolic issues. In fact, about 10% to 15% of people diagnosed with Type 2 diabetes are at a normal or low weight. This is often referred to as "lean diabetes."

### Actionable insight
- This CEs suggest that to maintain this low-risk status, it is essential to prevent high glucose index and the hypertension. Essentially, the model suggests that while the patient is currently safe, spikes in blood sugar and blood pressure are the fastest routes to a high stroke risk classification.



### 2/

### Original patient profile:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male | 58 | No | No | Yes | Private | Urban | 94.00 | 28.1 | Unknown | 1 |

### Counterfactuals generated:

| Gender | Age | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-----|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male | 55 | No | No | Yes | Self-employed | Urban | 93.67 | 29.3 | Unknown | 0 |
| Male | 56 | No | No | Yes | Private | Urban | 96.93 | 25.0 | smokes | 0 |
| Male | 57 | No | Yes | Yes | Self-employed | Urban | 92.82 | 27.8 | formerly smoked | 0 |

### Interpretation

1. **Target change:**  
   - All counterfactuals flip the prediction from **1 → 0** (stroke → non-stroke).

2. **Feature changes:**  
   - The most influential changes are:
     - **Age decrease** (from 58 → 55–57),
     - Slight variations in **average glucose level**,
     - Changes in **work type** and **smoking status**.
   - One counterfactual introduces **heart disease**, yet the prediction still flips to non-stroke.

3. **Unrealistic pattern:**  
   - The model predicts **lower stroke risk even with heart disease present**, while small reductions in age dominate the outcome.
   - Clinically, heart disease is a strong risk factor and should not be outweighed by such minor feature changes.



### 3/

**Original patient profile (idx:50):**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 57.0 | No           | No            | Yes     | Private   | Urban     | 95.40       | 19.5 | Unknown        | 1      |

**Counterfactuals generated:**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 45.0 | 0            | 0             | Yes     | Private   | Urban     | 97.95       | 24.5 | Unknown        | 0      |
| Female | 0.1  | 0            | 0             | Yes     | Private   | Urban     | 91.89       | 23.3 | Unknown        | 0      |
| Female | 0.1  | 0            | 0             | Yes     | Private   | Urban     | 83.09       | 24.5 | Unknown        | 0      |

### Interpretation

1. **Target change:**  
   All counterfactuals flip the predicted stroke from 1 → 0.  

2. **Feature changes:**  
   - The most significant changes are **age** and **BMI**, which decrease to much lower values.  
   - Hypertension, heart disease, work type, residence, and marriage status remain unchanged.  
   - Average glucose decreases slightly in some counterfactuals but remains relatively normal.  

3. **Insight:**  
   - The model heavily relies on **age** to predict stroke risk; extremely low age immediately flips the prediction to 0, even when BMI and glucose are elevated.  
   - This highlights that the model may overemphasize age relative to other risk factors.  
    

### Actionable insight

- Stroke risk in this scenario is driven primarily by **age**, with secondary influence from BMI and glucose.  
- **Extremely young counterfactuals** (0.1 years) illustrate that the model is highly influtential by age which isnot helpful in clinical insight.(i realized we should exclude this but it's too late too change).


### 4/

**Original patient profile (idx:100, FOr camparsion):**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 58.0 | No           | No            | Yes     | Private   | Urban     | 94.0        | 28.1 | Unknown        | 1      |

**Counterfactuals generated:**

| Gender | Age   | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|-------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 58.0  | 1            | 0             | Yes     | Private   | Urban     | 94.00       | 12.6 | Unknown        | 0      |
| Male   | 22.2  | 0            | 0             | Yes     | Govt_job  | Urban     | 94.00       | 28.1 | Unknown        | 0      |
| Male   | 58.0  | 0            | 0             | Yes     | Private   | Urban     | 65.92       | 28.1 | Unknown        | 0      |

### Interpretation

1. **Target change:**  
   All counterfactuals flip the predicted stroke from 1 → 0.  

2. **Feature changes:**  
   - **Hypertension** is added in one counterfactual.  
   - **Age** decreases dramatically in one counterfactual (58 → 22.2).  
   - **Average glucose** decreases in one counterfactual (94 → 65.92).  
   - **BMI** drops sharply in one counterfactual (28.1 → 12.6).  
   - Other features (gender, heart disease, marital status, work type, residence, smoking) remain unchanged.  

3. **Insight:**  
   - The model is highly sensitive to **age**, **BMI**, and **glucose** in predicting stroke risk.  
   - In some cases, **adding hypertension paradoxically reduces risk**, which may reflect **data skew**, where non-stroke cases with hypertension are overrepresented.  
 
### Actionable insight

- Stroke risk is influenced by a combination of (BMI, glucose) and age.  
- Clinically actionable interventions include:  
  - **Weight management** to reduce elevated BMI to healthy ranges  
  - **Glucose monitoring and control**, even if mildly elevated  
  - **Stress or lifestyle management** associated with work type or other contextual factors.  
- **Data skew caution:** The model may produce counterintuitive predictions (e.g., adding hypertension does not increase stroke risk) because most non-stroke cases in the dataset do not have heart disease.  



## CEs for Catboost

### 1/

**Original patient profile :**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 45.0 | No           | No            | Yes     | Govt_job  | Urban     | 55.47       | 19.8 | smokes         | 0      |

**Counterfactuals generated:**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type       | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------------|-----------|-------------|------|----------------|--------|
| Female | 45.0 | 0            | 0             | Yes     | Private         | Urban     | 72.65       | 25.6 | Unknown        | 1      |
| Female | 46.0 | 0            | 0             | Yes     | Self-employed   | Urban     | 71.12       | 27.3 | never smoked  | 1      |
| Female | 40.0 | 0            | 0             | Yes     | Private         | Urban     | 71.20       | 27.1 | never smoked  | 1      |

### Interpretation

1. **Target change:**  
   All counterfactuals flip the predicted stroke from 0 → 1.  

2. **Feature changes:**  
   - The most notable changes are **gender**, **BMI**, **average glucose level**, and **work type**.  
   - **Age remains approximately constant**, indicating that the prediction change is not driven by aging in this case.  
   - Hypertension and heart disease remain unchanged across all counterfactuals.  

3. **Insight:**  
   - The model assigns a higher stroke risk to **female profiles** compared to the original male profile, even at similar ages.  
   - Moderate increases in **BMI** and **average glucose**, while still within relatively normal ranges, are sufficient to flip the prediction.  
   - This suggests that the model captures **gender-dependent risk patterns** and is sensitive to relatively small metabolic changes when age is held constant.  

### Actionable insight

- These counterfactuals indicate that stroke risk is driven by a **combination of metabolic factors (BMI, glucose level)** and **lifestyle-related factors**, rather than age alone.  
- The change in **work type** (e.g., Govt_job → Private / Self-employed) suggests that **stress-related factors**—such as job pressure, workload, or employment instability—may play a role in increasing predicted stroke risk.  
- From a practical perspective, actionable recommendations should focus on **modifiable risk factors**, including:
  - Maintaining healthy **body weight (BMI)**  
  - Monitoring and controlling **blood glucose levels**, even when they are not in an extreme range  
  - Managing **work-related stress** through workload regulation, stress management strategies, or lifestyle adjustments  


### 2/

**Original patient profile :**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 69.0 | No           | Yes           | No      | Govt_job  | Urban     | 202.38      | 34.6 | Unknown        | 1      |

**Counterfactuals generated:**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 50.0 | 0            | 1             | Yes     | Private   | Urban     | 193.80      | 26.4 | never smoked  | 0      |
| Female | 52.0 | 0            | 0             | Yes     | Private   | Rural     | 200.46      | 25.0 | Unknown        | 0      |
| Female | 53.0 | 1            | 1             | Yes     | Private   | Urban     | 196.25      | 24.9 | smokes        | 0      |

### Interpretation

1. **Target change:**  
   All counterfactuals flip the predicted stroke from 1 → 0.  

2. **Feature changes:**  
   - The dominant change across all counterfactuals is **age**, which decreases by approximately 16–19 years.  
   - **Marriage status** and **work type** change consistently from *unmarried Govt_job* to *married Private employment*.  
   - **BMI decreases substantially**, moving from an obese range (34.6) to near-normal values (24.9–26.4).  
   - **Heart disease and hypertension** vary across counterfactuals, indicating secondary influence.  
   - Average glucose remains **extremely high** in all cases.  

3. **Insight:**  
   - The model predicts that a combination of **younger age**, **lower BMI**, and **changes in work context** can outweigh the effect of persistently high glucose levels.  
   - The shift from *Govt_job* to *Private employment* may implicitly capture **stress-related or lifestyle differences** that influence stroke risk.  
   - In some cases, active heart disease, the model assigns lower stroke risk to these profiles
   
### Actionable insight

- These counterfactuals suggest that **stroke risk is shaped by BMI and lifestyle-related factors**, rather than glucose levels alone.  
- Concrete, actionable recommendations derived from this analysis include:  
  - **Weight management** to reduce BMI from obese to healthy ranges  
  - **Stress-aware lifestyle**, particularly related to occupational conditions and work-related pressure  

- **Model caution:** In some counterfactuals, profiles with active heart disease are assigned lower stroke risk. This likely reflects a **skew in the data**, where a portion of non-stroke cases have heart disease. As a result, the model has learned a **spurious correlation**: “heart disease and hypertenstion → lower stroke probability.”  

### 3/

**Original patient profile (idx:50, For Comparasion):**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 57.0 | No           | No            | Yes     | Private   | Urban     | 95.40       | 19.5 | Unknown        | 1      |

**Counterfactuals generated:**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Female | 58.0 | 0            | 0             | Yes     | Private   | Urban     | 96.21       | 23.5 | never smoked  | 0      |
| Male   | 57.0 | 0            | 0             | Yes     | Private   | Rural     | 92.59       | 24.2 | Unknown        | 0      |
| Female | 54.0 | 0            | 0             | Yes     | Private   | Urban     | 99.83       | 22.7 | formerly smoked | 0      |

### Interpretation

1. **Target change:**  
   All counterfactuals flip the predicted stroke from 1 → 0.  

2. **Feature changes:**  
   - **BMI** increases slightly in two counterfactuals (19.5 → 22.7–23.5).  
   - **Age** varies slightly (54–58) in counterfactuals.  
   - **Gender** changes in one case (Female → Male).  
   - Minor variations in **average glucose** and **smoking status** are observed.  
   - Hypertension and heart disease remain unchanged.  

3. **Insight:**  
   - The model is sensitive to **BMI, age, and gender**  
   - The change from Female → Male suggests that Male has more chance to have stroke than Female(Checked by real clinical knowledge)  
   - Slight increases in BMI and changes in glucose show that the model considers even moderate variations in metabolic factors when determining stroke risk.  

### Actionable insight

- Stroke risk is influenced by (BMI, glucose), as well as lifestyle or stress-related context implied by work type and residence.  
- Concrete interventions could include:  
  - **Healthy weight maintenance** to optimize BMI  
  - **Monitoring blood glucose** even if values appear normal  
  - **Managing stress** associated with work or daily routines  



### 4/

**Original patient profile (idx:100, For Comaparasion):**

| Gender | Age  | Hypertension | Heart Disease | Married | Work Type | Residence | Avg Glucose | BMI  | Smoking Status | Target |
|--------|------|--------------|---------------|---------|-----------|-----------|-------------|------|----------------|--------|
| Male   | 58.0 | No           | No            | Yes     | Private   | Urban     | 94.0        | 28.1 | Unknown        | 1      |

**Counterfactuals generated:**

| Gender | Age   | Hypertension | Heart Disease | Married | Work Type     | Residence | Avg Glucose | BMI  | Smoking Status   | Target |
|--------|-------|--------------|---------------|---------|---------------|-----------|-------------|------|-----------------|--------|
| Male   | 55.0  | 0            | 0             | Yes     | Self-employed | Urban     | 93.67       | 29.3 | Unknown         | 0      |
| Male   | 56.0  | 0            | 0             | Yes     | Private       | Urban     | 96.93       | 25.0 | smokes          | 0      |
| Male   | 57.0  | 0            | 1             | Yes     | Self-employed | Urban     | 92.82       | 27.8 | formerly smoked | 0      |

### Interpretation

1. **Target change:**  
   All counterfactuals flip the predicted stroke from 1 → 0.  

2. **Feature changes:**  
   - **Age** decreases slightly in some counterfactuals (58 → 55–57).  
   - **BMI** varies moderately (25.0–29.3).  
   - **Heart disease** changes in one counterfactual, showing secondary influence.  
   - **Work type** changes (Private → Self-employed) in two counterfactuals, suggesting lifestyle or stress-related factors affect stroke prediction.  
   - Average glucose remains roughly constant (92–97), and other features remain unchanged.  

3. **Insight:**  
   - The model considers **age, BMI, and work type** as key factors for predicting stroke risk in this profile.  
   - Subtle differences in (BMI) and **contextual features** (work type) are sufficient to flip the predicted outcome.  
   - The heart disease change in one counterfactual illustrates that the data is skewed to no stroke .

### Actionable insight

- Stroke risk is influenced by (BMI) and **stress-related lifestyle factors** (work type).  
- Recommendations for this patient may include:  
  - **Weight management** to optimize BMI  
  - **Monitoring and controlling glucose levels**, even if moderately elevated  
  - **Stress reduction and lifestyle interventions**, particularly related to employment type or work conditions  
- **Data skew caution:** The model may assign lower stroke risk even when heart disease is present because most non-stroke cases in the training data do not have heart disease.  



## Summary of Counterfactual Explanations (CEs)

From analyzing multiple patient profiles and their counterfactuals, several clear patterns emerge regarding how the model predicts stroke risk:

1. **Age dominates the model predictions**  
   - Across nearly all CEs, **age is the most influential feature**.  
   - While this drives target changes effectively, it is **not clinically actionable**, as we cannot modify a patient’s age.  

2. **Metabolic factors (BMI and average glucose) are predictive when age is fixed**  
   - When age is held constant in counterfactual generation, the model often adjusts **BMI** or **average glucose** to flip predictions.  
   - Keeping these features within **normal or lower ranges** decreases the predicted stroke risk.  
   - These are **clinically actionable interventions**, such as weight management and glucose control.  

3. **Stress-related or lifestyle factors**  
   - Some CEs suggest that changing **work type** (e.g., Self-employed → Private) or **residence type** can reduce stroke risk.  
   - This aligns with real-world evidence: **stress and lifestyle influence cardiovascular health**.  

4. **Marriage appears protective**  
   - Interestingly, many CEs show that **being married** slightly lowers predicted stroke risk.  
   - A “fun insight”: **the power of love may be real**, at least according to the model’s learned patterns.  

5. **Challenges with actionable feature constraints**  
   - If **age and gender are fixed** (to focus on modifiable features), DiCE must rely on other variables like **BMI, glucose, or smoking** to flip the prediction.  
   - If these features have **low feature importance**, the Dice may **fail to generate a feasible counterfactual**, resulting in an **underconstrained problem**.  

6. **Detecting outliers**  
   - CEs are powerful for identifying **rare outliers or extreme feature combinations** that may not be obvious in standard analysis.  
   - This helps uncover **data issues or rare risk patterns** that could be clinically relevant.  

### Key Takeaways

- The **main actionable insights** from CEs are:  
  1. **Maintain healthy BMI**  
  2. **Control glucose levels**  
  3. **Address stress-related lifestyle factors** (work, residence, daily habits)  

- **Immutable features** like age and gender should generally be fixed to focus on clinically relevant interventions.  
- In small datasets where **age is extremely predictive**, the DiCE may struggle to find feasible counterfactuals if age is fixed.  
- Overall, CEs provide a **unique window into the model’s learned patterns**, offering both **actionable clinical guidance** and the ability to **detect outliers or unusual cases**.  

> **Conclusion:** Counterfactual explanations transform complex model behavior into **practical insights**, highlighting which features can be realistically modified to reduce stroke risk, while also revealing hidden patterns like stress, marriage, or outlier profiles.


# Partial Dependence Plots & Individual Conditional Expectation

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
%pip install catboost

features = ["age", "avg_glucose_level", "bmi"]

PartialDependenceDisplay.from_estimator(
    estimator=rf,
    X=X_test,
    features=features,
    kind="average",
    grid_resolution=50
)


In [ ]:
PartialDependenceDisplay.from_estimator(
    estimator=rf,
    X=X_test,
    features=features,
    kind="both",
    subsample=100,
    grid_resolution=50
)


In [ ]:
from sklearn.inspection import PartialDependenceDisplay

features = ["age", "avg_glucose_level", "bmi"]

PartialDependenceDisplay.from_estimator(
    estimator=xgb,          # 或 xgb
    X=X_test,
    features=features,
    kind="average",         # PDP
    grid_resolution=50
)


In [ ]:
PartialDependenceDisplay.from_estimator(
    estimator=xgb,
    X=X_test,
    features=features,
    kind="both",
    subsample=100,
    grid_resolution=50
)

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.inspection import PartialDependenceDisplay
from scipy.stats import chi2_contingency


required_cols = {"gender", "age", "avg_glucose_level", "bmi"}
missing = required_cols - set(X_test.columns)
if missing:
    raise ValueError(f"X_test is missing required columns: {missing}")


BMI_THRESHOLD = 30.0   # obesity cutoff (common choice)
high_bmi = X_test[X_test["bmi"] >= BMI_THRESHOLD].copy()

print(f"[Subgroup] High BMI (bmi >= {BMI_THRESHOLD}) size: {len(high_bmi)} / {len(X_test)}")

if len(high_bmi) < 50:
    print("Warning: High BMI subgroup is small; ICE/PDP may be noisy.")


In [ ]:
high_bmi_sorted = high_bmi.sort_values("avg_glucose_level")
low_glucose_person = high_bmi_sorted.iloc[[0]].copy()
high_glucose_person = high_bmi_sorted.iloc[[-1]].copy()

print("\n[Representative Individuals within High BMI]")
print("Low glucose person:")
display(low_glucose_person)
print("High glucose person:")
display(high_glucose_person)



In [ ]:

def plot_pdp_ice_for_subgroup(estimator, X_subgroup, feature="avg_glucose_level",
                             model_name="Model", subsample=100, grid_resolution=50):
    """
    Plots PDP + ICE for a single feature in a given subgroup.
    - PDP (average effect): thick line
    - ICE (individual effects): many thin lines (subsampled)
    """
    fig = plt.figure(figsize=(8, 5))
    PartialDependenceDisplay.from_estimator(
        estimator=estimator,
        X=X_subgroup,
        features=[feature],
        kind="both",                 # PDP + ICE
        subsample=subsample,         # number of ICE lines
        grid_resolution=grid_resolution,
        random_state=42
    )
    plt.title(f"{model_name}: PDP + ICE for '{feature}' (High BMI subgroup)")
    plt.show()


plot_pdp_ice_for_subgroup(
    estimator=rf,
    X_subgroup=high_bmi,
    feature="avg_glucose_level",
    model_name="Random Forest",
    subsample=100,
    grid_resolution=50
)


plot_pdp_ice_for_subgroup(
    estimator=xgb,
    X_subgroup=high_bmi,
    feature="avg_glucose_level",
    model_name="XGBoost",
    subsample=100,
    grid_resolution=50
)



In [ ]:

def compute_individual_ice(estimator, person_row: pd.DataFrame, feature: str, grid: np.ndarray):
    """
    For a single individual (one-row DataFrame), vary `feature` across `grid`,
    predict proba for class 1 (stroke), return probabilities.
    """
    if person_row.shape[0] != 1:
        raise ValueError("person_row must be a one-row DataFrame.")
    X_tmp = pd.concat([person_row] * len(grid), ignore_index=True)
    X_tmp[feature] = grid


    if hasattr(estimator, "predict_proba"):
        proba = estimator.predict_proba(X_tmp)[:, 1]
    else:

        if hasattr(estimator, "decision_function"):
            scores = estimator.decision_function(X_tmp)

            proba = 1 / (1 + np.exp(-scores))
        else:
            proba = estimator.predict(X_tmp)

    return proba

def plot_two_individual_ice(estimator, person_a, person_b, feature="avg_glucose_level",
                            model_name="Model", grid_resolution=80):
    """
    Plot ICE curves for two individuals by varying `feature`.
    Grid is constructed from subgroup min/max for interpretability.
    """

    fmin = float(high_bmi[feature].min())
    fmax = float(high_bmi[feature].max())
    grid = np.linspace(fmin, fmax, grid_resolution)

    proba_a = compute_individual_ice(estimator, person_a, feature, grid)
    proba_b = compute_individual_ice(estimator, person_b, feature, grid)

    plt.figure(figsize=(8, 5))
    plt.plot(grid, proba_a, label="High BMI + LOW glucose representative")
    plt.plot(grid, proba_b, label="High BMI + HIGH glucose representative")
    plt.xlabel(feature)
    plt.ylabel("Predicted P(stroke=1)")
    plt.title(f"{model_name}: Individual ICE curves for '{feature}' (High BMI subgroup)")
    plt.legend()
    plt.show()


plot_two_individual_ice(
    estimator=rf,
    person_a=low_glucose_person,
    person_b=high_glucose_person,
    feature="avg_glucose_level",
    model_name="Random Forest",
    grid_resolution=80
)


plot_two_individual_ice(
    estimator=xgb,
    person_a=low_glucose_person,
    person_b=high_glucose_person,
    feature="avg_glucose_level",
    model_name="XGBoost",
    grid_resolution=80
)



In [ ]:

print("\n[High BMI subgroup summary]")
print(high_bmi[["age", "avg_glucose_level", "bmi"]].describe())


plt.figure(figsize=(7, 4))
plt.hist(high_bmi["avg_glucose_level"], bins=30)
plt.title("avg_glucose_level distribution (High BMI subgroup)")
plt.xlabel("avg_glucose_level")
plt.ylabel("Count")
plt.show()

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.inspection import PartialDependenceDisplay
from scipy.stats import chi2_contingency



required_cols = {"gender", "age", "avg_glucose_level", "bmi"}
missing = required_cols - set(X_test.columns)
if missing:
    raise ValueError(f"X_test is missing required columns: {missing}")


high_bmi = X_test[X_test["bmi"] >= BMI_THRESHOLD].copy()

print(f"[Subgroup] High BMI (bmi >= {BMI_THRESHOLD}) size: {len(high_bmi)} / {len(X_test)}")

if len(high_bmi) < 50:
    print("Warning: High BMI subgroup is small; ICE/PDP may be noisy.")


high_bmi_sorted = high_bmi.sort_values("avg_glucose_level")
low_glucose_person = high_bmi_sorted.iloc[[0]].copy()
high_glucose_person = high_bmi_sorted.iloc[[-1]].copy()

print("\n[Representative Individuals within High BMI]")
print("Low glucose person:")
display(low_glucose_person)
print("High glucose person:")
display(high_glucose_person)


def plot_pdp_ice_for_subgroup(estimator, X_subgroup, feature="avg_glucose_level",
                             model_name="Model", subsample=100, grid_resolution=50):
    """
    Plots PDP + ICE for a single feature in a given subgroup.
    - PDP (average effect): thick line
    - ICE (individual effects): many thin lines (subsampled)
    """
    fig = plt.figure(figsize=(8, 5))
    PartialDependenceDisplay.from_estimator(
        estimator=estimator,
        X=X_subgroup,
        features=[feature],
        kind="both",
        subsample=subsample,
        grid_resolution=grid_resolution,
        random_state=42
    )
    plt.title(f"{model_name}: PDP + ICE for '{feature}' (High BMI subgroup)")
    plt.show()


plot_pdp_ice_for_subgroup(
    estimator=rf,
    X_subgroup=high_bmi,
    feature="avg_glucose_level",
    model_name="Random Forest",
    subsample=50,
    grid_resolution=50
)


plot_pdp_ice_for_subgroup(
    estimator=xgb,
    X_subgroup=high_bmi,
    feature="avg_glucose_level",
    model_name="XGBoost",
    subsample=50,
    grid_resolution=50
)


def compute_individual_ice(estimator, person_row: pd.DataFrame, feature: str, grid: np.ndarray):
    """
    For a single individual (one-row DataFrame), vary `feature` across `grid`,
    predict proba for class 1 (stroke), return probabilities.
    """
    if person_row.shape[0] != 1:
        raise ValueError("person_row must be a one-row DataFrame.")
    X_tmp = pd.concat([person_row] * len(grid), ignore_index=True)
    X_tmp[feature] = grid

    # Predict probability of class 1 if available; otherwise fall back to predict()
    if hasattr(estimator, "predict_proba"):
        proba = estimator.predict_proba(X_tmp)[:, 1]
    else:
        # Some estimators may not implement predict_proba; use decision_function or predict
        if hasattr(estimator, "decision_function"):
            scores = estimator.decision_function(X_tmp)
            # Convert scores to pseudo-prob via sigmoid (rough)
            proba = 1 / (1 + np.exp(-scores))
        else:
            proba = estimator.predict(X_tmp)

    return proba

def plot_two_individual_ice(estimator, person_a, person_b, feature="avg_glucose_level",
                            model_name="Model", grid_resolution=80):
    """
    Plot ICE curves for two individuals by varying `feature`.
    Grid is constructed from subgroup min/max for interpretability.
    """
    # Build grid from high_bmi subgroup range for this feature
    fmin = float(high_bmi[feature].min())
    fmax = float(high_bmi[feature].max())
    grid = np.linspace(fmin, fmax, grid_resolution)

    proba_a = compute_individual_ice(estimator, person_a, feature, grid)
    proba_b = compute_individual_ice(estimator, person_b, feature, grid)

    plt.figure(figsize=(8, 5))
    plt.plot(grid, proba_a, label="High BMI + LOW glucose representative")
    plt.plot(grid, proba_b, label="High BMI + HIGH glucose representative")
    plt.xlabel(feature)
    plt.ylabel("Predicted P(stroke=1)")
    plt.title(f"{model_name}: Individual ICE curves for '{feature}' (High BMI subgroup)")
    plt.legend()
    plt.show()

# Individual ICE curves for Random Forest
plot_two_individual_ice(
    estimator=rf,
    person_a=low_glucose_person,
    person_b=high_glucose_person,
    feature="avg_glucose_level",
    model_name="Random Forest",
    grid_resolution=80
)

# Individual ICE curves for XGBoost
plot_two_individual_ice(
    estimator=xgb,
    person_a=low_glucose_person,
    person_b=high_glucose_person,
    feature="avg_glucose_level",
    model_name="XGBoost",
    grid_resolution=80
)

# -----------------------------
# 5) (Optional) Quick subgroup stats to help you describe the subgroup
# -----------------------------
print("\n[High BMI subgroup summary]")
print(high_bmi[["age", "avg_glucose_level", "bmi"]].describe())

# (Optional) If you want to see the distribution of glucose within high BMI
plt.figure(figsize=(7, 4))
plt.hist(high_bmi["avg_glucose_level"], bins=30)
plt.title("avg_glucose_level distribution (High BMI subgroup)")
plt.xlabel("avg_glucose_level")
plt.ylabel("Count")
plt.show()


In [ ]:
BMI_THRESHOLD = 30.0

high_bmi = X_test[X_test["bmi"] >= BMI_THRESHOLD].copy()

high_bmi_female = high_bmi[high_bmi["gender"] == 0]
high_bmi_male   = high_bmi[high_bmi["gender"] == 1]

print("High BMI female:", len(high_bmi_female))
print("High BMI male:", len(high_bmi_male))


In [ ]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

def plot_gender_specific_pdp_ice(estimator, X_subgroup, gender_name,
                                 subsample=20, grid_resolution=50):
    PartialDependenceDisplay.from_estimator(
        estimator=estimator,
        X=X_subgroup,
        features=["avg_glucose_level"],
        kind="both",                 # PDP + ICE
        subsample=subsample,
        grid_resolution=grid_resolution,
        random_state=42
    )
    plt.title(f"{gender_name} | High BMI | avg_glucose_level")
    plt.show()

# Random Forest
plot_gender_specific_pdp_ice(
    estimator=rf,
    X_subgroup=high_bmi_female,
    gender_name="Female (High BMI)",
    subsample=50
)

plot_gender_specific_pdp_ice(
    estimator=rf,
    X_subgroup=high_bmi_male,
    gender_name="Male (High BMI)",
    subsample=50
)


In [ ]:
# XGBoost
plot_gender_specific_pdp_ice(
    estimator=xgb,
    X_subgroup=high_bmi_female,
    gender_name="Female (High BMI)",
    subsample=50
)

plot_gender_specific_pdp_ice(
    estimator=xgb,
    X_subgroup=high_bmi_male,
    gender_name="Male (High BMI)",
    subsample=50
)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay

BMI_THRESHOLD = 30.0
high_bmi = X_test[X_test["bmi"] >= BMI_THRESHOLD].copy()

age_bins = [0, 40, 60, 120]
age_labels = ["<40", "40-60", ">=60"]
high_bmi["age_group"] = pd.cut(high_bmi["age"], bins=age_bins, labels=age_labels, right=False)

print("High BMI size:", len(high_bmi))
print(high_bmi["age_group"].value_counts(dropna=False))
print(high_bmi["gender"].value_counts(dropna=False))


In [ ]:
gender_map = {0: "Female", 1: "Male"}
for g in sorted(high_bmi["gender"].dropna().unique()):
    for ag in age_labels:
        X_sub = high_bmi[(high_bmi["gender"] == g) & (high_bmi["age_group"] == ag)]
        plot_glucose_pdp_ice(
            estimator=rf,
            X_sub=X_sub,
            title=f"RF | High BMI | {gender_map.get(g, g)} | Age {ag}",
            subsample=20
        )


In [ ]:
def plot_glucose_pdp_ice(estimator, X_sub, title, subsample=20, grid_resolution=50):
    if len(X_sub) < 20:
        print(f"Skip (too small): {title} | n={len(X_sub)}")
        return
    
    X_sub_model = X_sub.drop(columns=["age_group"], errors="ignore")

    PartialDependenceDisplay.from_estimator(
        estimator=estimator,
        X=X_sub_model,
        features=["avg_glucose_level"],
        kind="both",
        subsample=subsample,
        grid_resolution=grid_resolution,
        random_state=42
    )
    plt.title(f"{title} | n={len(X_sub)}")
    plt.show()
print("len(X_sub) =", len(X_sub))
print("columns =", list(X_sub.columns)[:15], "...")
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay

plot_glucose_pdp_ice(rf, X_sub, "debug", subsample=10)



In [ ]:
for g in sorted(high_bmi["gender"].dropna().unique()):
    for ag in age_labels:
        X_sub = high_bmi[(high_bmi["gender"] == g) & (high_bmi["age_group"] == ag)]
        plot_glucose_pdp_ice(
            estimator=xgb,
            X_sub=X_sub,
            title=f"XGB | High BMI | {gender_map.get(g, g)} | Age {ag}",
            subsample=20
        )
